In [ ]:
import pandas as pd
import numpy as np
from datetime import datetime, timedelta
import ipywidgets as widgets
from IPython.display import display, clear_output, HTML
import io
import base64
import os
from pathlib import Path
import json
from itertools import product
import warnings
warnings.filterwarnings('ignore')

# ============================================================================
# GLOBAL STATE MANAGEMENT
# ============================================================================

class BacktestState:
    """Centralized state management for the backtest system"""
    def __init__(self):
        self.uploaded_files = {}  # {filename: dataframe}
        self.processed_data = {}  # {filename: processed_dataframe}
        self.strategy_configs = {}  # {strategy_name: config_df}
        self.all_results = {}  # {strategy_name: {filename: results}}
        self.consolidated_results = None
        self.folder_path = None
        self.entry_time_override = None  # User-specified entry time
        self.exit_dte_override = None  # User-specified exit DTE (REQUIREMENT 3)
        self.exit_time_override = None  # User-specified exit time (REQUIREMENT 3)
        
    def reset(self):
        """Reset all state"""
        self.__init__()
        
    def add_file(self, filename, content):
        """Add uploaded file"""
        self.uploaded_files[filename] = content
        
    def get_file_count(self):
        """Get count of uploaded files"""
        return len(self.uploaded_files)
    
    def get_filenames(self):
        """Get list of uploaded filenames"""
        return list(self.uploaded_files.keys())
    
    def add_strategy_config(self, strategy_name, config):
        """Add a strategy configuration"""
        self.strategy_configs[strategy_name] = config
    
    def get_strategy_names(self):
        """Get list of configured strategies"""
        return list(self.strategy_configs.keys())

# Initialize global state
state = BacktestState()

# ============================================================================
# STRATEGY PRESETS - DTE-specific values (5 DTEs: 4,3,2,1,0)
# ============================================================================

STRATEGY_PRESETS = {
    "Conservative": {
        "description": "Lower risk with tighter stops and wider strikes, adjusting as expiry approaches",
        # Parameters per DTE: [4 DTE, 3 DTE, 2 DTE, 1 DTE, 0 DTE]
        "stoploss_pct": [45, 45, 50, 50, 55],
        "profit_target_pct": [65, 65, 60, 60, 55],
        "pe_strike_offset": [7, 7, 6, 6, 5],
        "ce_strike_offset": [7, 7, 6, 6, 5],
        "index_movement": [70, 75, 80, 85, 90],
        "lot_size": [1, 1, 1, 1, 1]
    },
    "Moderate": {
        "description": "Balanced approach with standard parameters, slight adjustments across DTEs",
        "stoploss_pct": [60, 60, 60, 65, 65],
        "profit_target_pct": [50, 50, 50, 45, 45],
        "pe_strike_offset": [4, 4, 4, 3, 3],
        "ce_strike_offset": [4, 4, 4, 3, 3],
        "index_movement": [100, 110, 115, 120, 125],
        "lot_size": [1, 1, 1, 1, 1]
    },
    "Aggressive": {
        "description": "Higher risk with wider stops and tighter strikes, maximizing premium capture",
        "stoploss_pct": [75, 75, 80, 80, 85],
        "profit_target_pct": [45, 45, 40, 40, 35],
        "pe_strike_offset": [4, 4, 3, 3, 2],
        "ce_strike_offset": [4, 4, 3, 3, 2],
        "index_movement": [120, 125, 130, 135, 140],
        "lot_size": [1, 1, 1, 1, 1]
    },
    "Scalper": {
        "description": "Quick profits with very tight targets and stops across all DTEs",
        "stoploss_pct": [30, 30, 35, 35, 40],
        "profit_target_pct": [35, 35, 30, 30, 25],
        "pe_strike_offset": [6, 6, 5, 5, 4],
        "ce_strike_offset": [6, 6, 5, 5, 4],
        "index_movement": [50, 55, 60, 65, 70],
        "lot_size": [1, 1, 1, 1, 1]
    },
    "Custom": {
        "description": "User-defined parameters with standard defaults",
        "stoploss_pct": [60, 60, 60, 50, 50],
        "profit_target_pct": [50, 50, 50, 50, 50],
        "pe_strike_offset": [4, 4, 4, 4, 4],
        "ce_strike_offset": [4, 4, 4, 4, 4],
        "index_movement": [100, 100, 200, 150, 100],
        "lot_size": [1, 1, 1, 1, 1]
    }
}

# DTE labels for display - MUST match preset array order (highest to lowest)
DTE_LABELS = ['4 DTE', '3 DTE', '2 DTE', '1 DTE', '0 DTE']

# ============================================================================
# DATA PREPROCESSING
# ============================================================================

def preprocess_data(df):
    """Enhanced data preprocessing matching your original logic"""
    
    # Display settings
    pd.set_option('display.max_columns', None)
    pd.set_option('display.max_rows', 100)
    
    # Ensure required columns exist
    required_cols = ['expiry_date', 'date_only', 'time', 'symbol', 'open', 'high', 'low', 'close', 'index_close']
    missing_cols = [col for col in required_cols if col not in df.columns]
    
    if missing_cols:
        raise ValueError(f"Missing required columns: {missing_cols}")
    
    # Convert date columns
    df['expiry_date'] = pd.to_datetime(df['expiry_date']).dt.date
    df['date_only'] = pd.to_datetime(df['date_only']).dt.date
    
    # Calculate days to expiry
    df['days_to_expiry'] = (pd.to_datetime(df['expiry_date']) - pd.to_datetime(df['date_only'])).dt.days
    
    # Filter for weekly options (DTE <= 6)
    df = df[df['days_to_expiry'] <= 6].copy()
    
    # Select required columns - include DTE column
    if 'DTE' in df.columns:
        backtest_data = df[['expiry_date', "date_only", 'days_to_expiry', 'time', 'symbol', 
                            'open', 'high', 'low', 'close', 'index_close', 'DTE']].copy()
    else:
        backtest_data = df[['expiry_date', "date_only", 'days_to_expiry', 'time', 'symbol', 
                            'open', 'high', 'low', 'close', 'index_close']].copy()
    
    # Extract dte_format from DTE column if available (e.g., "5DTE" -> "5 DTE", "ODTE" -> "0 DTE")
    if 'DTE' in backtest_data.columns:
        def extract_dte_format(dte_str):
            """Extract DTE format from string like '5DTE', '3DTE', 'ODTE'"""
            if pd.isna(dte_str):
                return None
            
            dte_str = str(dte_str).strip().upper()
            
            # Handle ODTE specially
            if dte_str == 'ODTE' or dte_str == '0DTE':
                return '0 DTE'
            
            # Extract first number from string
            import re
            match = re.search(r'(\d+)', dte_str)
            if match:
                dte_number = match.group(1)
                return f'{dte_number} DTE'
            
            return None
        
        backtest_data['dte_format'] = backtest_data['DTE'].apply(extract_dte_format)
        
        # Verify dte_format was created successfully
        if backtest_data['dte_format'].isna().all() or backtest_data['dte_format'].isna().any():
            print("⚠️  Warning: Could not extract dte_format from DTE column for all rows. Falling back to days_to_expiry.")
            # Fallback to days_to_expiry based mapping
            backtest_data['dte_format'] = backtest_data['days_to_expiry'].apply(lambda x: f'{x} DTE')
    else:
        # No DTE column, use days_to_expiry directly
        print("ℹ️  Info: DTE column not found. Using days_to_expiry for dte_format.")
        backtest_data['dte_format'] = backtest_data['days_to_expiry'].apply(lambda x: f'{x} DTE')
    
    return backtest_data


# ============================================================================
# FILE HANDLING - OPTIMIZED
# ============================================================================

def load_files_from_folder(folder_path):
    """
    Load all CSV and Excel files from a folder path
    Returns: dict of {filename: dataframe}
    """
    loaded_files = {}
    folder = Path(folder_path)
    
    if not folder.exists():
        raise ValueError(f"Folder path does not exist: {folder_path}")
    
    # Supported file extensions
    supported_extensions = ['.csv', '.xlsx', '.xls']
    
    # Get all files
    files = [f for f in folder.iterdir() if f.suffix.lower() in supported_extensions]
    
    if not files:
        raise ValueError(f"No CSV or Excel files found in: {folder_path}")
    
    print(f"📂 Found {len(files)} files in folder")
    
    for file_path in files:
        try:
            filename = file_path.name
            print(f"   Loading: {filename}...", end=" ")
            
            if file_path.suffix.lower() == '.csv':
                df = pd.read_csv(file_path)
            else:
                df = pd.read_excel(file_path)
            
            loaded_files[filename] = df
            print(f"✅ ({len(df)} rows)")
            
        except Exception as e:
            print(f"❌ Error: {str(e)}")
            continue
    
    return loaded_files

# ============================================================================
# STRATEGY CONFIGURATION GENERATOR
# ============================================================================

def create_base_strategy_config(backtest_data, force_standard_dtes=True):
    """
    Create base strategy configuration
    
    Args:
        backtest_data: Preprocessed backtest data
        force_standard_dtes: If True, always include 4,3,2,1,0 DTE regardless of data
    
    Returns:
        DataFrame with strategy configuration
    """
    
    if force_standard_dtes:
        # REQUIREMENT 1: Always include standard DTEs (4, 3, 2, 1, 0)
        # This ensures all files generate consistent strategy configs
        standard_dtes = ['4 DTE', '3 DTE', '2 DTE', '1 DTE', '0 DTE']
        
        # Base configuration with standard DTEs
        base_config = pd.DataFrame({
            'DTE': standard_dtes,
            'Stoploss %': [60, 60, 60, 50, 50],
            'Profit Target %': [50, 50, 50, 50, 50],
            'lot_size': [1, 1, 1, 1, 1],
            'PE Strike': [4, 4, 4, 4, 4],
            'CE Strike': [4, 4, 4, 4, 4],
            'Index Movement +/-': [100, 100, 200, 150, 100]
        })
        
        # Get index_close at 9:15 for each DTE (if available in data)
        def get_index_close_at_915(dte):
            filtered = backtest_data[
                (backtest_data['dte_format'] == dte) &
                (backtest_data['time'] == '09:15')
            ]
            if not filtered.empty:
                return filtered['index_close'].iloc[0]
            # If DTE doesn't exist in this file, use a placeholder
            # It will be calculated dynamically during execution
            return 23000.0  # Default placeholder
        
        base_config['Index Close'] = base_config['DTE'].apply(get_index_close_at_915)
        base_config['Index Close'] = (base_config['Index Close'] / 50).round() * 50
        
        print(f"✅ Standard strategy config created with DTEs: {standard_dtes}")
        
    else:
        # Original behavior: Use DTEs from data
        unique_dte = sorted(backtest_data['dte_format'].dropna().unique(), reverse=True)
        
        # Base configuration
        base_config = pd.DataFrame({
            'DTE': unique_dte,
            'Stoploss %': [60, 60, 60, 60, 60, 50, 50][:len(unique_dte)],
            'Profit Target %': [50, 50, 50, 50, 50, 50, 50][:len(unique_dte)],
            'lot_size': [1, 1, 1, 1, 1, 1, 1][:len(unique_dte)],
            'PE Strike': [4, 4, 4, 4, 4, 4, 4][:len(unique_dte)],
            'CE Strike': [4, 4, 4, 4, 4, 4, 4][:len(unique_dte)],
            'Index Movement +/-': [100, 100, 100, 200, 200, 150, 100][:len(unique_dte)]
        })
        
        # Get index_close at 9:15 for each DTE
        def get_index_close_at_915(dte):
            filtered = backtest_data[
                (backtest_data['dte_format'] == dte) &
                (backtest_data['time'] == '09:15')
            ]
            if not filtered.empty:
                return filtered['index_close'].iloc[0]
            return np.nan
        
        base_config['Index Close'] = base_config['DTE'].apply(get_index_close_at_915)
        base_config['Index Close'] = (base_config['Index Close'] / 50).round() * 50
    
    return base_config

def apply_preset_to_config(base_config, preset_params):
    """
    Apply preset parameters to configuration with DTE-specific values
    preset_params should contain arrays: stoploss_pct, profit_target_pct, pe_strike_offset, 
                                          ce_strike_offset, index_movement, lot_size
    Each array should have values for each DTE in the base_config
    """
    
    modified_config = base_config.copy()
    num_dtes = len(modified_config)
    
    # Ensure we have the right number of values for each parameter
    # If preset has fewer values than DTEs, repeat the last value
    def extend_to_length(arr, length):
        if len(arr) >= length:
            return arr[:length]
        else:
            return arr + [arr[-1]] * (length - len(arr))
    
    # Apply DTE-specific values
    modified_config['Stoploss %'] = extend_to_length(preset_params['stoploss_pct'], num_dtes)
    modified_config['Profit Target %'] = extend_to_length(preset_params['profit_target_pct'], num_dtes)
    modified_config['PE Strike'] = extend_to_length(preset_params['pe_strike_offset'], num_dtes)
    modified_config['CE Strike'] = extend_to_length(preset_params['ce_strike_offset'], num_dtes)
    modified_config['Index Movement +/-'] = extend_to_length(preset_params['index_movement'], num_dtes)
    modified_config['lot_size'] = extend_to_length(preset_params['lot_size'], num_dtes)
    
    return modified_config

# ============================================================================
# REQUIREMENT 2: EXCEL-BASED STRATEGY CONFIGURATION
# ============================================================================

def load_strategies_from_excel(excel_file_path):
    """
    Load strategy configurations from Excel file
    
    Expected Excel format:
    | Strategy | DTE | Stoploss % | Profit Target % | lot_size | PE Strike | CE Strike | Index Movement +/- |
    
    Args:
        excel_file_path: Path to Excel file or file-like object
    
    Returns:
        dict: {strategy_name: config_df}
    """
    try:
        # Read Excel file
        df = pd.read_excel(excel_file_path)
        
        # Validate required columns
        required_columns = ['Strategy', 'DTE', 'Stoploss %', 'Profit Target %', 
                           'lot_size', 'PE Strike', 'CE Strike', 'Index Movement +/-']
        
        missing_cols = [col for col in required_columns if col not in df.columns]
        if missing_cols:
            raise ValueError(f"Missing required columns: {missing_cols}")
        
        # Group by Strategy
        strategies = {}
        for strategy_name, group in df.groupby('Strategy'):
            # Sort by DTE descending (4, 3, 2, 1, 0)
            group = group.sort_values('DTE', ascending=False).copy()
            
            # Create config dataframe
            config = pd.DataFrame({
                'DTE': group['DTE'].apply(lambda x: f'{int(x)} DTE'),
                'Stoploss %': group['Stoploss %'].astype(int),
                'Profit Target %': group['Profit Target %'].astype(int),
                'lot_size': group['lot_size'].astype(int),
                'PE Strike': group['PE Strike'].astype(int),
                'CE Strike': group['CE Strike'].astype(int),
                'Index Movement +/-': group['Index Movement +/-'].astype(int),
                'Index Close': 23000.0  # Placeholder, will be calculated
            })
            
            strategies[strategy_name] = config.reset_index(drop=True)
        
        print(f"✅ Loaded {len(strategies)} strategies from Excel: {list(strategies.keys())}")
        return strategies
        
    except Exception as e:
        print(f"❌ Error loading Excel file: {str(e)}")
        raise

def validate_excel_strategy_config(strategies_dict):
    """
    Validate Excel-loaded strategy configurations
    
    Args:
        strategies_dict: Dictionary of strategy configs
    
    Returns:
        tuple: (is_valid, error_messages)
    """
    errors = []
    
    for strategy_name, config in strategies_dict.items():
        # Check if all required DTEs are present (4, 3, 2, 1, 0)
        expected_dtes = ['4 DTE', '3 DTE', '2 DTE', '1 DTE', '0 DTE']
        actual_dtes = config['DTE'].tolist()
        
        missing_dtes = [dte for dte in expected_dtes if dte not in actual_dtes]
        if missing_dtes:
            errors.append(f"{strategy_name}: Missing DTEs {missing_dtes}")
        
        # Check value ranges
        if (config['Stoploss %'] < 0).any() or (config['Stoploss %'] > 100).any():
            errors.append(f"{strategy_name}: Stoploss % must be between 0-100")
        
        if (config['Profit Target %'] < 0).any() or (config['Profit Target %'] > 100).any():
            errors.append(f"{strategy_name}: Profit Target % must be between 0-100")
        
        if (config['PE Strike'] < 0).any() or (config['CE Strike'] < 0).any():
            errors.append(f"{strategy_name}: Strike offsets must be positive")
        
        if (config['lot_size'] < 1).any():
            errors.append(f"{strategy_name}: Lot size must be >= 1")
    
    is_valid = len(errors) == 0
    return is_valid, errors

def create_excel_template():
    """
    Create an Excel template for strategy configuration
    
    Returns:
        DataFrame: Template with sample data
    """
    template_data = {
        'Strategy': ['Conservative', 'Conservative', 'Conservative', 'Conservative', 'Conservative',
                     'Aggressive', 'Aggressive', 'Aggressive', 'Aggressive', 'Aggressive'],
        'DTE': [4, 3, 2, 1, 0, 4, 3, 2, 1, 0],
        'Stoploss %': [45, 45, 50, 50, 55, 75, 75, 80, 80, 85],
        'Profit Target %': [65, 65, 60, 60, 55, 45, 45, 40, 40, 35],
        'lot_size': [1, 1, 1, 1, 1, 1, 1, 1, 1, 1],
        'PE Strike': [7, 7, 6, 6, 5, 4, 4, 3, 3, 2],
        'CE Strike': [7, 7, 6, 6, 5, 4, 4, 3, 3, 2],
        'Index Movement +/-': [70, 75, 80, 85, 90, 120, 125, 130, 135, 140]
    }
    
    return pd.DataFrame(template_data)


def generate_strategy_combinations(base_config, selected_presets_with_params):
    """
    Generate strategy configurations for selected presets with their parameters
    
    Args:
        base_config: Base configuration DataFrame
        selected_presets_with_params: dict of {strategy_name: preset_params}
    
    Returns: 
        dict of {strategy_name: config_df}
    """
    strategies = {}
    
    for strategy_name, preset_params in selected_presets_with_params.items():
        config = apply_preset_to_config(base_config, preset_params)
        strategies[strategy_name] = config
    
    return strategies

def create_custom_strategy(base_config, custom_params):
    """
    Create a custom strategy configuration
    custom_params: dict with direct parameter values
    """
    custom_config = base_config.copy()
    num_dtes = len(custom_config)
    
    if 'stoploss_pct' in custom_params:
        custom_config['Stoploss %'] = custom_params['stoploss_pct'] if isinstance(custom_params['stoploss_pct'], list) else [custom_params['stoploss_pct']] * num_dtes
    if 'profit_target_pct' in custom_params:
        custom_config['Profit Target %'] = custom_params['profit_target_pct'] if isinstance(custom_params['profit_target_pct'], list) else [custom_params['profit_target_pct']] * num_dtes
    if 'pe_strike_offset' in custom_params:
        custom_config['PE Strike'] = custom_params['pe_strike_offset'] if isinstance(custom_params['pe_strike_offset'], list) else [custom_params['pe_strike_offset']] * num_dtes
    if 'ce_strike_offset' in custom_params:
        custom_config['CE Strike'] = custom_params['ce_strike_offset'] if isinstance(custom_params['ce_strike_offset'], list) else [custom_params['ce_strike_offset']] * num_dtes
    if 'index_movement' in custom_params:
        custom_config['Index Movement +/-'] = custom_params['index_movement'] if isinstance(custom_params['index_movement'], list) else [custom_params['index_movement']] * num_dtes
    if 'lot_size' in custom_params:
        custom_config['lot_size'] = custom_params['lot_size'] if isinstance(custom_params['lot_size'], list) else [custom_params['lot_size']] * num_dtes
    
    return custom_config

# ============================================================================
# CORE STRATEGY EXECUTION (WITH CRITICAL FIX APPLIED)
# ============================================================================

def execute_strategy(strategy_row, backtest_data, strategy_config, filename, strategy_name):
    """
    Execute strategy for a given DTE configuration with carry forward and re-entry logic
    
    ⚡ CRITICAL FIX APPLIED (December 2025):
    Strike calculation now uses actual index close from each file's data at entry time,
    not pre-configured strategy values. This ensures each expiry file independently
    derives PE and CE strikes based on its own market conditions.
    """
    
    initial_dte = strategy_row['DTE']
    
    # Track all trades
    all_trades = []
    
    # Get all unique DTEs in descending order
    all_dtes = sorted(backtest_data['dte_format'].dropna().unique(), reverse=True)
    
    # Start from initial DTE
    current_dte_index = all_dtes.index(initial_dte)
    current_date = None
    current_time = None
    last_exit_index_close = None
    continue_trading = True
    trade_number = 1
    
    # Trading loop
    while continue_trading and current_dte_index < len(all_dtes):
        
        # Get current DTE for entry
        entry_dte = all_dtes[current_dte_index]
        
        # ═══════════════════════════════════════════════════════════════════
        # ERROR HANDLING: Check if DTE exists in strategy config
        # ═══════════════════════════════════════════════════════════════════
        matching_strategy = strategy_config[strategy_config['DTE'] == entry_dte]
        
        if matching_strategy.empty:
            error_msg = (
                f"\n{'='*80}\n"
                f"❌ ERROR: Missing DTE Configuration\n"
                f"{'='*80}\n"
                f"Strategy: {strategy_name}\n"
                f"File: {filename}\n"
                f"Missing DTE: {entry_dte}\n"
                f"\n"
                f"Available DTEs in strategy config:\n"
                f"{strategy_config['DTE'].tolist()}\n"
                f"\n"
                f"DTEs found in data file:\n"
                f"{all_dtes}\n"
                f"\n"
                f"SOLUTION:\n"
                f"1. Your strategy config doesn't have parameters for '{entry_dte}'\n"
                f"2. Edit strategy parameters to include ALL DTEs present in your data\n"
                f"3. OR select a different Entry DTE that exists in the strategy\n"
                f"{'='*80}\n"
            )
            raise ValueError(error_msg)
        
        entry_strategy = matching_strategy.iloc[0]
        # ═══════════════════════════════════════════════════════════════════
        
        # Get entry parameters
        pe_strike_offset = entry_strategy['PE Strike']
        ce_strike_offset = entry_strategy['CE Strike']
        lot_size = entry_strategy['lot_size']
        
        # ═══════════════════════════════════════════════════════════════════
        # ⚡ CRITICAL FIX: DYNAMIC STRIKE CALCULATION PER FILE
        # ═══════════════════════════════════════════════════════════════════
        # OLD CODE (FAULTY):
        #   if last_exit_index_close is not None:
        #       index_close_for_entry = last_exit_index_close
        #   else:
        #       index_close_for_entry = entry_strategy['Index Close']  # ❌ WRONG
        #
        # PROBLEM: Used pre-configured value from strategy config, which was
        # calculated once from first file and reused for all files, causing
        # duplicate strikes across different expiry files.
        #
        # NEW CODE (FIXED):
        # Now fetches actual index close from each file's data at entry time
        # ═══════════════════════════════════════════════════════════════════
        
        if last_exit_index_close is not None:
            # Re-entry scenario: Use index close from last exit (unchanged)
            index_close_for_entry = last_exit_index_close
        else:
            # First entry scenario: Get ACTUAL index close from THIS FILE's data
            # Filter data for this DTE
            dte_data = backtest_data[backtest_data['dte_format'] == entry_dte].copy()
            
            # Get entry time from state if overridden, otherwise use default '09:15'
            entry_time_to_use = getattr(state, 'entry_time_override', '09:15') or '09:15'
            
            # Get the first entry point at specified time
            temp_entry_data = dte_data[dte_data['time'] == entry_time_to_use]
            
            if not temp_entry_data.empty:
                # ✅ USE ACTUAL INDEX CLOSE FROM THIS FILE
                index_close_for_entry = temp_entry_data.iloc[0]['index_close']
                print(f"📍 {filename} | {entry_dte} | Index Close at {entry_time_to_use}: {index_close_for_entry}")
            else:
                # Fallback to strategy config (should rarely happen)
                index_close_for_entry = entry_strategy['Index Close']
                print(f"⚠️  {filename} | {entry_dte} | Using fallback Index Close: {index_close_for_entry}")
        
        # Round to nearest 50
        index_close_for_entry = round(index_close_for_entry / 50) * 50
        
        # ═══════════════════════════════════════════════════════════════════
        # END OF CRITICAL FIX
        # ═══════════════════════════════════════════════════════════════════
        
        # Calculate actual strike prices based on THIS FILE's index close
        pe_strike = index_close_for_entry - (pe_strike_offset * 50)
        ce_strike = index_close_for_entry + (ce_strike_offset * 50)
        
        print(f"🎯 {filename} | {entry_dte} | Strikes: PE={pe_strike}, CE={ce_strike}")
        
        # Create symbol patterns
        pe_symbol = f"{int(pe_strike)}-PE"
        ce_symbol = f"{int(ce_strike)}-CE"
        
        # Filter backtest data for this DTE
        dte_data = backtest_data[backtest_data['dte_format'] == entry_dte].copy()
        
        # Get entry point
        if current_date is None and current_time is None:
            # First entry at user-specified time (or default 09:15)
            entry_time_to_use = getattr(state, 'entry_time_override', '09:15') or '09:15'
            
            pe_entry = dte_data[
                (dte_data['symbol'].str.contains(pe_symbol, na=False)) &
                (dte_data['time'] == entry_time_to_use)
            ]
            ce_entry = dte_data[
                (dte_data['symbol'].str.contains(ce_symbol, na=False)) &
                (dte_data['time'] == entry_time_to_use)
            ]
        else:
            # Re-entry logic
            pe_candidates = dte_data[
                (dte_data['symbol'].str.contains(pe_symbol, na=False))
            ].copy()
            
            ce_candidates = dte_data[
                (dte_data['symbol'].str.contains(ce_symbol, na=False))
            ].copy()
            
            if pe_candidates.empty or ce_candidates.empty:
                current_dte_index += 1
                current_date = None
                current_time = None
                last_exit_index_close = None
                continue
            
            # Create datetime for filtering
            pe_candidates['datetime_check'] = pd.to_datetime(
                pe_candidates['date_only'].astype(str) + ' ' + pe_candidates['time'].astype(str)
            )
            ce_candidates['datetime_check'] = pd.to_datetime(
                ce_candidates['date_only'].astype(str) + ' ' + ce_candidates['time'].astype(str)
            )
            
            exit_datetime_str = f"{current_date} {current_time}"
            exit_datetime = pd.to_datetime(exit_datetime_str)
            
            # Filter for rows AFTER exit time
            pe_entry = pe_candidates[pe_candidates['datetime_check'] > exit_datetime].copy()
            ce_entry = ce_candidates[ce_candidates['datetime_check'] > exit_datetime].copy()
            
            if pe_entry.empty or ce_entry.empty:
                current_dte_index += 1
                current_date = None
                current_time = None
                last_exit_index_close = None
                continue
            
            pe_entry = pe_entry.iloc[[0]]
            ce_entry = ce_entry.iloc[[0]]
            
            pe_entry = pe_entry.drop(columns=['datetime_check'])
            ce_entry = ce_entry.drop(columns=['datetime_check'])
        
        if pe_entry.empty or ce_entry.empty:
            break
        
        # Get entry prices and details
        pe_entry_price = pe_entry.iloc[0]['close']
        ce_entry_price = ce_entry.iloc[0]['close']
        total_entry_price = pe_entry_price + ce_entry_price
        entry_date = pe_entry.iloc[0]['date_only']
        entry_time = pe_entry.iloc[0]['time']
        entry_index_close = pe_entry.iloc[0]['index_close']
        
        # Monitor across DTEs until exit
        trade_exited = False
        
        for dte_idx in range(current_dte_index, len(all_dtes)):
            current_monitoring_dte = all_dtes[dte_idx]
            
            # Get strategy parameters for current monitoring DTE
            current_strategy = strategy_config[strategy_config['DTE'] == current_monitoring_dte].iloc[0]
            current_stoploss_pct = current_strategy['Stoploss %']
            current_profit_target_pct = current_strategy['Profit Target %']
            current_index_movement = current_strategy['Index Movement +/-']
            
            # Calculate current targets
            stoploss_price = total_entry_price * (1 + current_stoploss_pct/100)
            profit_target_price = total_entry_price * (1 - current_profit_target_pct/100)
            index_upper_limit = entry_index_close + current_index_movement
            index_lower_limit = entry_index_close - current_index_movement
            
            # Get DTE data
            dte_data = backtest_data[backtest_data['dte_format'] == current_monitoring_dte].copy()
            
            # Filter monitoring data
            pe_monitoring = dte_data[
                (dte_data['symbol'].str.contains(pe_symbol, na=False))
            ].copy()
            
            ce_monitoring = dte_data[
                (dte_data['symbol'].str.contains(ce_symbol, na=False))
            ].copy()
            
            if pe_monitoring.empty or ce_monitoring.empty:
                continue
            
            # Create datetime column for proper time filtering
            pe_monitoring['datetime_check'] = pd.to_datetime(
                pe_monitoring['date_only'].astype(str) + ' ' + pe_monitoring['time'].astype(str)
            )
            ce_monitoring['datetime_check'] = pd.to_datetime(
                ce_monitoring['date_only'].astype(str) + ' ' + ce_monitoring['time'].astype(str)
            )
            
            # Create entry datetime for filtering
            entry_datetime = pd.to_datetime(f"{entry_date} {entry_time}")
            
            # Filter for data >= entry time
            pe_monitoring = pe_monitoring[pe_monitoring['datetime_check'] >= entry_datetime].copy()
            ce_monitoring = ce_monitoring[ce_monitoring['datetime_check'] >= entry_datetime].copy()
            
            if pe_monitoring.empty or ce_monitoring.empty:
                continue
            
            # Drop datetime_check before merge
            pe_monitoring = pe_monitoring.drop(columns=['datetime_check'])
            ce_monitoring = ce_monitoring.drop(columns=['datetime_check'])
            
            # Merge PE and CE data
            monitoring_data = pd.merge(
                pe_monitoring[['date_only', 'time', 'close', 'index_close']],
                ce_monitoring[['date_only', 'time', 'close']],
                on=['date_only', 'time'],
                suffixes=('_pe', '_ce')
            )
            
            if monitoring_data.empty:
                continue
            
            # Calculate combined position value
            monitoring_data['total_close'] = monitoring_data['close_pe'] + monitoring_data['close_ce']
            
            # Check for exit conditions
            for idx, row in monitoring_data.iterrows():
                # Skip entry time point
                if row['date_only'] == entry_date and row['time'] == entry_time:
                    continue
                
                # Check if 0 DTE and time is 13:20 (FINAL EXIT)
                if current_monitoring_dte == '0 DTE' and row['time'] == '13:20':
                    exit_time = row['time']
                    exit_date = row['date_only']
                    exit_price = row['total_close']
                    exit_index_close = row['index_close']
                    pnl = (total_entry_price - exit_price) * lot_size * 50
                    
                    trade_result = {
                        'Strategy': strategy_name,
                        'Filename': filename,
                        'Trade_Number': trade_number,
                        'Entry_DTE': entry_dte,
                        'Exit_DTE': current_monitoring_dte,
                        'Entry_Date': entry_date,
                        'Entry_Time': entry_time,
                        'Exit_Date': exit_date,
                        'Exit_Time': exit_time,
                        'PE_Strike': pe_strike,
                        'CE_Strike': ce_strike,
                        'PE_Entry_Price': pe_entry_price,
                        'CE_Entry_Price': ce_entry_price,
                        'Total_Entry_Price': total_entry_price,
                        'Exit_Price': exit_price,
                        'Exit_Reason': '0 DTE Expiry (3:20 PM)',
                        'PnL': pnl,
                        'Entry_Index_Close': entry_index_close,
                        'Entry_Index_Close_at_Entry': index_close_for_entry,
                        'Exit_Index_Close': exit_index_close
                    }
                    
                    all_trades.append(trade_result)
                    continue_trading = False
                    trade_exited = True
                    break
                
                # Check stoploss breach
                if row['total_close'] >= stoploss_price:
                    exit_time = row['time']
                    exit_date = row['date_only']
                    exit_price = row['total_close']
                    exit_index_close = row['index_close']
                    pnl = (total_entry_price - exit_price) * lot_size * 50
                    
                    trade_result = {
                        'Strategy': strategy_name,
                        'Filename': filename,
                        'Trade_Number': trade_number,
                        'Entry_DTE': entry_dte,
                        'Exit_DTE': current_monitoring_dte,
                        'Entry_Date': entry_date,
                        'Entry_Time': entry_time,
                        'Exit_Date': exit_date,
                        'Exit_Time': exit_time,
                        'PE_Strike': pe_strike,
                        'CE_Strike': ce_strike,
                        'PE_Entry_Price': pe_entry_price,
                        'CE_Entry_Price': ce_entry_price,
                        'Total_Entry_Price': total_entry_price,
                        'Exit_Price': exit_price,
                        'Exit_Reason': 'Stoploss Hit',
                        'PnL': pnl,
                        'Entry_Index_Close': entry_index_close,
                        'Entry_Index_Close_at_Entry': index_close_for_entry,
                        'Exit_Index_Close': exit_index_close
                    }
                    
                    all_trades.append(trade_result)
                    
                    current_date = exit_date
                    current_time = exit_time
                    last_exit_index_close = exit_index_close
                    current_dte_index = all_dtes.index(current_monitoring_dte)
                    trade_exited = True
                    trade_number += 1
                    break
                
                # Check profit target breach
                if row['total_close'] <= profit_target_price:
                    exit_time = row['time']
                    exit_date = row['date_only']
                    exit_price = row['total_close']
                    exit_index_close = row['index_close']
                    pnl = (total_entry_price - exit_price) * lot_size * 50
                    
                    trade_result = {
                        'Strategy': strategy_name,
                        'Filename': filename,
                        'Trade_Number': trade_number,
                        'Entry_DTE': entry_dte,
                        'Exit_DTE': current_monitoring_dte,
                        'Entry_Date': entry_date,
                        'Entry_Time': entry_time,
                        'Exit_Date': exit_date,
                        'Exit_Time': exit_time,
                        'PE_Strike': pe_strike,
                        'CE_Strike': ce_strike,
                        'PE_Entry_Price': pe_entry_price,
                        'CE_Entry_Price': ce_entry_price,
                        'Total_Entry_Price': total_entry_price,
                        'Exit_Price': exit_price,
                        'Exit_Reason': 'Profit Target Hit',
                        'PnL': pnl,
                        'Entry_Index_Close': entry_index_close,
                        'Entry_Index_Close_at_Entry': index_close_for_entry,
                        'Exit_Index_Close': exit_index_close
                    }
                    
                    all_trades.append(trade_result)
                    
                    current_date = exit_date
                    current_time = exit_time
                    last_exit_index_close = exit_index_close
                    current_dte_index = all_dtes.index(current_monitoring_dte)
                    trade_exited = True
                    trade_number += 1
                    break
                
                # Check index movement breach
                if row['index_close'] >= index_upper_limit or row['index_close'] <= index_lower_limit:
                    exit_time = row['time']
                    exit_date = row['date_only']
                    exit_price = row['total_close']
                    exit_index_close = row['index_close']
                    pnl = (total_entry_price - exit_price) * lot_size * 50
                    
                    trade_result = {
                        'Strategy': strategy_name,
                        'Filename': filename,
                        'Trade_Number': trade_number,
                        'Entry_DTE': entry_dte,
                        'Exit_DTE': current_monitoring_dte,
                        'Entry_Date': entry_date,
                        'Entry_Time': entry_time,
                        'Exit_Date': exit_date,
                        'Exit_Time': exit_time,
                        'PE_Strike': pe_strike,
                        'CE_Strike': ce_strike,
                        'PE_Entry_Price': pe_entry_price,
                        'CE_Entry_Price': ce_entry_price,
                        'Total_Entry_Price': total_entry_price,
                        'Exit_Price': exit_price,
                        'Exit_Reason': 'Index Movement Breach',
                        'PnL': pnl,
                        'Entry_Index_Close': entry_index_close,
                        'Entry_Index_Close_at_Entry': index_close_for_entry,
                        'Exit_Index_Close': exit_index_close
                    }
                    
                    all_trades.append(trade_result)
                    
                    current_date = exit_date
                    current_time = exit_time
                    last_exit_index_close = exit_index_close
                    current_dte_index = all_dtes.index(current_monitoring_dte)
                    trade_exited = True
                    trade_number += 1
                    break
            
            if trade_exited:
                break
        
        if not trade_exited:
            break
    
    return all_trades

# ============================================================================
# MULTI-STRATEGY BACKTEST EXECUTOR
# ============================================================================

def run_multi_strategy_backtest(files_dict, strategies_dict, progress_callback=None):
    """
    Run backtest across multiple files and strategies
    
    Args:
        files_dict: {filename: dataframe}
        strategies_dict: {strategy_name: config_df}
        progress_callback: function to report progress
    
    Returns:
        all_results: {strategy_name: {filename: [trades]}}
    """
    all_results = {}
    total_combinations = len(files_dict) * len(strategies_dict)
    current_combination = 0
    
    for strategy_name, strategy_config in strategies_dict.items():
        all_results[strategy_name] = {}
        
        for filename, df in files_dict.items():
            current_combination += 1
            
            if progress_callback:
                progress_callback(f"Processing {strategy_name} on {filename} ({current_combination}/{total_combinations})")
            
            try:
                # Process data if not already processed
                if filename not in state.processed_data:
                    backtest_data = preprocess_data(df)
                    state.processed_data[filename] = backtest_data
                else:
                    backtest_data = state.processed_data[filename]
                
                # Execute strategy
                max_dte_strategy = strategy_config.iloc[0]
                results = execute_strategy(max_dte_strategy, backtest_data, strategy_config, filename, strategy_name)
                
                all_results[strategy_name][filename] = results
                
            except Exception as e:
                print(f"❌ Error: {strategy_name} on {filename}: {str(e)}")
                all_results[strategy_name][filename] = []
    
    return all_results

# ============================================================================
# RESULTS CONSOLIDATION & ANALYSIS
# ============================================================================

def consolidate_results(all_results):
    """
    Consolidate all results into a single DataFrame
    
    Args:
        all_results: {strategy_name: {filename: [trades]}}
    
    Returns:
        consolidated_df: DataFrame with all trades
    """
    all_trades = []
    
    for strategy_name, file_results in all_results.items():
        for filename, trades in file_results.items():
            all_trades.extend(trades)
    
    if not all_trades:
        return None
    
    return pd.DataFrame(all_trades)

def calculate_strategy_metrics(results_df, strategy_name):
    """Calculate comprehensive metrics for a strategy"""
    
    strategy_df = results_df[results_df['Strategy'] == strategy_name]
    
    if len(strategy_df) == 0:
        return None
    
    total_pnl = strategy_df['PnL'].sum()
    avg_pnl = strategy_df['PnL'].mean()
    winning_trades = len(strategy_df[strategy_df['PnL'] > 0])
    losing_trades = len(strategy_df[strategy_df['PnL'] < 0])
    win_rate = (winning_trades / len(strategy_df) * 100) if len(strategy_df) > 0 else 0
    
    avg_win = strategy_df[strategy_df['PnL'] > 0]['PnL'].mean() if winning_trades > 0 else 0
    avg_loss = strategy_df[strategy_df['PnL'] < 0]['PnL'].mean() if losing_trades > 0 else 0
    
    profit_factor = (strategy_df[strategy_df['PnL'] > 0]['PnL'].sum() / 
                     abs(strategy_df[strategy_df['PnL'] < 0]['PnL'].sum())) if losing_trades > 0 else float('inf')
    
    # Drawdown calculation
    strategy_sorted = strategy_df.sort_values(['Filename', 'Entry_Date', 'Entry_Time']).copy()
    strategy_sorted['Cumulative_PnL'] = strategy_sorted['PnL'].cumsum()
    strategy_sorted['Running_Max'] = strategy_sorted['Cumulative_PnL'].cummax()
    strategy_sorted['Drawdown'] = strategy_sorted['Cumulative_PnL'] - strategy_sorted['Running_Max']
    max_drawdown = strategy_sorted['Drawdown'].min()
    
    # Sharpe ratio (simplified)
    sharpe = (avg_pnl / strategy_df['PnL'].std()) if strategy_df['PnL'].std() > 0 else 0
    
    return {
        'Strategy': strategy_name,
        'Total_Trades': len(strategy_df),
        'Winning_Trades': winning_trades,
        'Losing_Trades': losing_trades,
        'Win_Rate_%': round(win_rate, 2),
        'Total_PnL': round(total_pnl, 2),
        'Avg_PnL': round(avg_pnl, 2),
        'Avg_Win': round(avg_win, 2),
        'Avg_Loss': round(avg_loss, 2),
        'Best_Trade': round(strategy_df['PnL'].max(), 2),
        'Worst_Trade': round(strategy_df['PnL'].min(), 2),
        'Profit_Factor': round(profit_factor, 2) if profit_factor != float('inf') else 'Inf',
        'Max_Drawdown': round(max_drawdown, 2),
        'Sharpe_Ratio': round(sharpe, 2),
        'Files_Processed': strategy_df['Filename'].nunique()
    }

def generate_strategy_comparison(results_df):
    """Generate comparison metrics across all strategies"""
    
    strategies = results_df['Strategy'].unique()
    comparison_data = []
    
    for strategy in strategies:
        metrics = calculate_strategy_metrics(results_df, strategy)
        if metrics:
            comparison_data.append(metrics)
    
    comparison_df = pd.DataFrame(comparison_data)
    
    # Rank strategies
    comparison_df['Rank_by_PnL'] = comparison_df['Total_PnL'].rank(ascending=False)
    comparison_df['Rank_by_WinRate'] = comparison_df['Win_Rate_%'].rank(ascending=False)
    comparison_df['Rank_by_Sharpe'] = comparison_df['Sharpe_Ratio'].rank(ascending=False)
    comparison_df['Rank_by_ProfitFactor'] = comparison_df['Profit_Factor'].apply(
        lambda x: float(x) if x != 'Inf' else 999
    ).rank(ascending=False)
    
    # Overall score (lower is better)
    comparison_df['Overall_Rank_Score'] = (
        comparison_df['Rank_by_PnL'] + 
        comparison_df['Rank_by_WinRate'] + 
        comparison_df['Rank_by_Sharpe'] + 
        comparison_df['Rank_by_ProfitFactor']
    ) / 4
    
    comparison_df = comparison_df.sort_values('Overall_Rank_Score')
    
    return comparison_df

# ============================================================================
# EXCEL EXPORT WITH COMPREHENSIVE ANALYSIS
# ============================================================================

def export_comprehensive_excel(results_df, strategies_dict, output_filename):
    """
    Export comprehensive Excel report with all strategies and analysis
    """
    
    print(f"\n📊 Generating comprehensive Excel report: {output_filename}")
    
    with pd.ExcelWriter(output_filename, engine='openpyxl') as writer:
        
        # Sheet 1: Strategy Comparison (THE MOST IMPORTANT)
        comparison_df = generate_strategy_comparison(results_df)
        comparison_df.to_excel(writer, sheet_name='Strategy_Comparison', index=False)
        print("   ✓ Sheet 1: Strategy_Comparison (Rankings)")
        
        # Sheet 2: All Trades
        all_trades_export = results_df.copy()
        all_trades_export['Week_Number'] = pd.to_datetime(all_trades_export['Entry_Date']).dt.isocalendar().week
        all_trades_export['Month'] = pd.to_datetime(all_trades_export['Entry_Date']).dt.to_period('M').astype(str)
        all_trades_export = all_trades_export.sort_values(['Strategy', 'Filename', 'Entry_Date', 'Entry_Time'])
        all_trades_export.to_excel(writer, sheet_name='All_Trades', index=False)
        print("   ✓ Sheet 2: All_Trades")
        
        # Sheet 3: Strategy Configs Used
        config_rows = []
        for strategy_name, config_df in strategies_dict.items():
            config_with_name = config_df.copy()
            config_with_name.insert(0, 'Strategy', strategy_name)
            config_rows.append(config_with_name)
        
        all_configs = pd.concat(config_rows, ignore_index=True)
        all_configs.to_excel(writer, sheet_name='Strategy_Configurations', index=False)
        print("   ✓ Sheet 3: Strategy_Configurations")
        
        # Sheet 4-N: Individual Strategy Details
        for idx, strategy_name in enumerate(results_df['Strategy'].unique(), 4):
            strategy_df = results_df[results_df['Strategy'] == strategy_name]
            sheet_name = f"{strategy_name[:25]}_Details"  # Excel sheet name limit
            
            # Add cumulative P&L
            strategy_sorted = strategy_df.sort_values(['Filename', 'Entry_Date', 'Entry_Time']).copy()
            strategy_sorted['Cumulative_PnL'] = strategy_sorted['PnL'].cumsum()
            
            strategy_sorted.to_excel(writer, sheet_name=sheet_name, index=False)
            print(f"   ✓ Sheet {idx}: {sheet_name}")
        
        # Exit Reason Analysis (Combined)
        exit_analysis = results_df.groupby(['Strategy', 'Exit_Reason']).agg({
            'PnL': ['sum', 'mean', 'count']
        }).round(2)
        exit_analysis.columns = ['Total_PnL', 'Avg_PnL', 'Count']
        exit_analysis = exit_analysis.reset_index()
        exit_analysis.to_excel(writer, sheet_name='Exit_Reason_Analysis', index=False)
        print(f"   ✓ Sheet {idx+1}: Exit_Reason_Analysis")
        
        # DTE Analysis (Combined)
        dte_analysis = results_df.groupby(['Strategy', 'Entry_DTE']).agg({
            'PnL': ['sum', 'mean', 'std', 'count']
        }).round(2)
        dte_analysis.columns = ['Total_PnL', 'Avg_PnL', 'Std_Dev', 'Count']
        dte_analysis = dte_analysis.reset_index()
        dte_analysis.to_excel(writer, sheet_name='DTE_Analysis', index=False)
        print(f"   ✓ Sheet {idx+2}: DTE_Analysis")
    
    print(f"\n✅ Excel report generated successfully!")
    return output_filename

# ============================================================================
# ENHANCED UI COMPONENTS
# ============================================================================

def create_enhanced_file_upload_ui():
    """Create enhanced file upload interface with folder path option"""
    
    # File upload widget
    upload_widget = widgets.FileUpload(
        accept='.csv,.xlsx,.xls',
        multiple=True,
        description='Upload Files',
        button_style='primary',
        layout=widgets.Layout(width='300px')
    )
    
    # Folder path input
    folder_path_input = widgets.Text(
        value='',
        placeholder='Enter folder path (e.g., /path/to/data)',
        description='Folder Path:',
        layout=widgets.Layout(width='500px')
    )
    
    load_folder_btn = widgets.Button(
        description='📂 Load from Folder',
        button_style='info',
        layout=widgets.Layout(width='200px')
    )
    
    file_list_output = widgets.Output()
    upload_status = widgets.HTML()
    
    def on_upload_change(change):
        """Handle file upload"""
        with file_list_output:
            clear_output()
            
            uploaded = change['new']
            if not uploaded:
                return
            
            for file_info in uploaded:
                try:
                    filename = file_info['name']
                    content = file_info['content']
                    
                    if filename.endswith('.csv'):
                        df = pd.read_csv(io.BytesIO(content))
                    elif filename.endswith(('.xlsx', '.xls')):
                        df = pd.read_excel(io.BytesIO(content))
                    else:
                        print(f"❌ Unsupported file format: {filename}")
                        continue
                    
                    state.add_file(filename, df)
                    print(f"✅ Uploaded: {filename} ({len(df)} rows)")
                    
                except Exception as e:
                    print(f"❌ Error uploading {filename}: {str(e)}")
            
            upload_status.value = f'<p style="color: green;">📁 {state.get_file_count()} files uploaded successfully</p>'
    
    def on_load_folder_click(b):
        """Handle folder path loading"""
        with file_list_output:
            clear_output()
            
            folder_path = folder_path_input.value.strip()
            if not folder_path:
                print("❌ Please enter a folder path")
                return
            
            try:
                loaded_files = load_files_from_folder(folder_path)
                
                for filename, df in loaded_files.items():
                    state.add_file(filename, df)
                
                upload_status.value = f'<p style="color: green;">📁 {len(loaded_files)} files loaded from folder</p>'
                state.folder_path = folder_path
                
            except Exception as e:
                print(f"❌ Error loading from folder: {str(e)}")
    
    upload_widget.observe(on_upload_change, names='value')
    load_folder_btn.on_click(on_load_folder_click)
    
    return widgets.VBox([
        widgets.HTML('<h3 style="color: #2c3e50;">📁 Step 1: Load Data Files</h3>'),
        widgets.HTML('<p style="color: #7f8c8d;">Choose one of the following options:</p>'),
        widgets.HTML('<p><strong>Option A:</strong> Upload files individually</p>'),
        upload_widget,
        widgets.HTML('<p><strong>Option B:</strong> Load all files from a folder</p>'),
        widgets.HBox([folder_path_input, load_folder_btn]),
        upload_status,
        file_list_output
    ])

def create_strategy_selection_ui():
    """Create enhanced strategy selection and configuration UI with DTE-specific editing capability"""
    
    # Strategy selection checkboxes
    preset_checkboxes = {}
    strategy_config_widgets = {}  # Store config widgets for each strategy and DTE
    
    # Preview and configuration area
    preview_output = widgets.Output()
    config_edit_output = widgets.Output()
    
    # Buttons
    select_all_btn = widgets.Button(
        description='✓ Select All',
        button_style='',
        layout=widgets.Layout(width='120px')
    )
    
    deselect_all_btn = widgets.Button(
        description='✗ Deselect All',
        button_style='',
        layout=widgets.Layout(width='120px')
    )
    
    preview_btn = widgets.Button(
        description='👁️ Preview Selected Strategies',
        button_style='info',
        layout=widgets.Layout(width='250px')
    )
    
    edit_btn = widgets.Button(
        description='✏️ Edit Strategy Parameters (DTE-Specific)',
        button_style='warning',
        layout=widgets.Layout(width='300px')
    )
    
    generate_btn = widgets.Button(
        description='🔧 Generate Strategy Configs',
        button_style='success',
        layout=widgets.Layout(width='250px')
    )
    
    config_output = widgets.Output()
    
    # Create checkboxes for each preset
    for preset_name, preset_info in STRATEGY_PRESETS.items():
        preset_checkboxes[preset_name] = widgets.Checkbox(
            value=False,
            description=f"{preset_name}",
            layout=widgets.Layout(width='200px')
        )
        
        # Store DTE-specific widgets for each strategy
        # Assuming 7 DTEs (6, 5, 4, 3, 2, 1, 0)
        num_dtes = len(preset_info['stoploss_pct'])
        
        strategy_config_widgets[preset_name] = {}
        for dte_idx in range(num_dtes):
            dte_label = DTE_LABELS[dte_idx] if dte_idx < len(DTE_LABELS) else f'{dte_idx} DTE'
            strategy_config_widgets[preset_name][dte_label] = {
                'stoploss': widgets.IntText(
                    value=preset_info['stoploss_pct'][dte_idx], 
                    layout=widgets.Layout(width='70px')
                ),
                'profit': widgets.IntText(
                    value=preset_info['profit_target_pct'][dte_idx], 
                    layout=widgets.Layout(width='70px')
                ),
                'pe_strike': widgets.IntText(
                    value=preset_info['pe_strike_offset'][dte_idx], 
                    layout=widgets.Layout(width='70px')
                ),
                'ce_strike': widgets.IntText(
                    value=preset_info['ce_strike_offset'][dte_idx], 
                    layout=widgets.Layout(width='70px')
                ),
                'index_move': widgets.IntText(
                    value=preset_info['index_movement'][dte_idx], 
                    layout=widgets.Layout(width='70px')
                ),
                'lot_size': widgets.IntText(
                    value=preset_info['lot_size'][dte_idx], 
                    layout=widgets.Layout(width='70px')
                )
            }
    
    def on_select_all(b):
        for checkbox in preset_checkboxes.values():
            checkbox.value = True
    
    def on_deselect_all(b):
        for checkbox in preset_checkboxes.values():
            checkbox.value = False
    
    def on_preview_click(b):
        """Preview selected strategies with their DTE-specific parameters"""
        with preview_output:
            clear_output()
            
            selected = [name for name, checkbox in preset_checkboxes.items() if checkbox.value]
            
            if not selected:
                print("❌ Please select at least one strategy!")
                return
            
            print("="*100)
            print("📊 SELECTED STRATEGIES PREVIEW - DTE-SPECIFIC PARAMETERS")
            print("="*100)
            print(f"\nTotal Strategies Selected: {len(selected)}\n")
            
            for idx, strategy_name in enumerate(selected, 1):
                preset_info = STRATEGY_PRESETS[strategy_name]
                params_widgets = strategy_config_widgets[strategy_name]
                
                print(f"\n{'═'*100}")
                print(f"{idx}. {strategy_name}")
                print(f"   Description: {preset_info['description']}")
                print(f"{'═'*100}")
                
                # Create table header
                print(f"\n{'DTE':<10} {'Stop Loss %':<15} {'Profit %':<15} {'PE Strike':<12} {'CE Strike':<12} {'Index Move':<12} {'Lot Size':<10}")
                print("─" * 100)
                
                # Show values for each DTE
                for dte_label in DTE_LABELS[:len(params_widgets)]:
                    dte_params = params_widgets[dte_label]
                    print(f"{dte_label:<10} "
                          f"{dte_params['stoploss'].value:<15} "
                          f"{dte_params['profit'].value:<15} "
                          f"{dte_params['pe_strike'].value:<12} "
                          f"{dte_params['ce_strike'].value:<12} "
                          f"±{dte_params['index_move'].value:<11} "
                          f"{dte_params['lot_size'].value:<10}")
                
                print()
            
            print("\n" + "="*100)
            print("✅ Preview complete. Click 'Edit Strategy Parameters' to modify individual DTEs.")
            print("="*100)
    
    def on_edit_click(b):
        """Show editable parameter widgets for selected strategies with DTE-specific controls"""
        with config_edit_output:
            clear_output()
            
            selected = [name for name, checkbox in preset_checkboxes.items() if checkbox.value]
            
            if not selected:
                print("❌ Please select at least one strategy first!")
                return
            
            print("="*100)
            print("✏️  EDIT STRATEGY PARAMETERS - DTE-SPECIFIC CONFIGURATION")
            print("="*100)
            print("\nModify parameters for each DTE (Days To Expiry) below:\n")
            
            for strategy_name in selected:
                params_widgets = strategy_config_widgets[strategy_name]
                
                print(f"\n{'═'*100}")
                display(widgets.HTML(f"<h3 style='color: #2c3e50; margin: 10px 0;'>{strategy_name}</h3>"))
                display(widgets.HTML(f"<p style='color: #7f8c8d; margin: 5px 0;'>{STRATEGY_PRESETS[strategy_name]['description']}</p>"))
                print(f"{'═'*100}\n")
                
                # Create table header with labels
                header_html = """
                <div style='background: #f8f9fa; padding: 10px; border-radius: 5px; margin-bottom: 10px;'>
                    <table style='width: 100%; text-align: center; font-weight: bold;'>
                        <tr>
                            <td style='width: 12%;'>DTE</td>
                            <td style='width: 14%;'>Stop Loss %</td>
                            <td style='width: 14%;'>Profit %</td>
                            <td style='width: 14%;'>PE Strike</td>
                            <td style='width: 14%;'>CE Strike</td>
                            <td style='width: 16%;'>Index Move</td>
                            <td style='width: 14%;'>Lot Size</td>
                        </tr>
                    </table>
                </div>
                """
                display(widgets.HTML(header_html))
                
                # Display editable widgets for each DTE in a table-like format
                for dte_label in DTE_LABELS[:len(params_widgets)]:
                    dte_params = params_widgets[dte_label]
                    
                    # Label for DTE
                    dte_label_widget = widgets.Label(
                        value=dte_label,
                        layout=widgets.Layout(width='100px')
                    )
                    
                    # Create row with all parameters
                    row = widgets.HBox([
                        dte_label_widget,
                        dte_params['stoploss'],
                        dte_params['profit'],
                        dte_params['pe_strike'],
                        dte_params['ce_strike'],
                        dte_params['index_move'],
                        dte_params['lot_size']
                    ], layout=widgets.Layout(margin='2px 0'))
                    
                    display(row)
                
                print()
            
            print("\n" + "="*100)
            print("✅ Edit complete. Changes save automatically.")
            print("💡 Tip: Click 'Preview' to see updated values in table format.")
            print("🔧 Click 'Generate' when ready to create configurations.")
            print("="*100)
    
    def on_generate_click(b):
        """Generate strategy configurations using current DTE-specific parameter values"""
        with config_output:
            clear_output()
            
            if state.get_file_count() == 0:
                print("❌ Please upload files first!")
                return
            
            # Get selected presets
            selected = [name for name, checkbox in preset_checkboxes.items() if checkbox.value]
            
            if not selected:
                print("❌ Please select at least one strategy preset!")
                return
            
            print(f"🔧 Generating DTE-specific configurations for {len(selected)} strategies...")
            
            # Process first file to get DTE structure
            first_filename = state.get_filenames()[0]
            first_df = state.uploaded_files[first_filename]
            
            try:
                processed = preprocess_data(first_df)
                state.processed_data[first_filename] = processed
                
                # Create base config
                base_config = create_base_strategy_config(processed)
                
                # Get actual DTEs from base_config (sorted descending: 6, 5, 4, 3, 2, 1, 0)
                actual_dtes_in_config = base_config['DTE'].tolist()
                
                print(f"📊 DTEs found in data: {actual_dtes_in_config}")
                
                # Prepare parameters dict for each selected strategy
                selected_presets_with_params = {}
                for strategy_name in selected:
                    params_widgets = strategy_config_widgets[strategy_name]
                    
                    # ═══════════════════════════════════════════════════════════════
                    # CRITICAL FIX: Map widget values to actual DTEs in config
                    # ═══════════════════════════════════════════════════════════════
                    # Widget keys are like '6 DTE', '5 DTE', etc. (from DTE_LABELS)
                    # We need to extract values in the order of actual_dtes_in_config
                    
                    stoploss_list = []
                    profit_list = []
                    pe_strike_list = []
                    ce_strike_list = []
                    index_move_list = []
                    lot_size_list = []
                    
                    for actual_dte in actual_dtes_in_config:
                        # Find the matching widget key for this DTE
                        widget_key = actual_dte  # e.g., '6 DTE', '3 DTE'
                        
                        if widget_key in params_widgets:
                            dte_params = params_widgets[widget_key]
                            stoploss_list.append(dte_params['stoploss'].value)
                            profit_list.append(dte_params['profit'].value)
                            pe_strike_list.append(dte_params['pe_strike'].value)
                            ce_strike_list.append(dte_params['ce_strike'].value)
                            index_move_list.append(dte_params['index_move'].value)
                            lot_size_list.append(dte_params['lot_size'].value)
                        else:
                            # DTE exists in data but not in widget config - use defaults
                            print(f"⚠️  Warning: {strategy_name} missing config for {actual_dte}, using defaults")
                            stoploss_list.append(60)
                            profit_list.append(50)
                            pe_strike_list.append(4)
                            ce_strike_list.append(4)
                            index_move_list.append(100)
                            lot_size_list.append(1)
                    # ═══════════════════════════════════════════════════════════════
                    
                    selected_presets_with_params[strategy_name] = {
                        'stoploss_pct': stoploss_list,
                        'profit_target_pct': profit_list,
                        'pe_strike_offset': pe_strike_list,
                        'ce_strike_offset': ce_strike_list,
                        'index_movement': index_move_list,
                        'lot_size': lot_size_list
                    }
                
                # Generate strategy combinations
                strategies = generate_strategy_combinations(base_config, selected_presets_with_params)
                
                # Store in state
                for strategy_name, config in strategies.items():
                    state.add_strategy_config(strategy_name, config)
                
                print(f"\n✅ Generated {len(strategies)} strategy configurations with DTE-specific parameters!")
                
                print("\n" + "="*100)
                print("📋 STRATEGY CONFIGURATIONS SUMMARY")
                print("="*100)
                
                # Show summary for each strategy
                for idx, strategy_name in enumerate(strategies.keys(), 1):
                    print(f"\n{idx}. {strategy_name}")
                    print("   " + "─"*80)
                    params = selected_presets_with_params[strategy_name]
                    
                    # Show parameter ranges
                    print(f"   Stop Loss Range:     {min(params['stoploss_pct'])}% - {max(params['stoploss_pct'])}%")
                    print(f"   Profit Target Range: {min(params['profit_target_pct'])}% - {max(params['profit_target_pct'])}%")
                    print(f"   PE Strike Range:     {min(params['pe_strike_offset'])} - {max(params['pe_strike_offset'])}")
                    print(f"   CE Strike Range:     {min(params['ce_strike_offset'])} - {max(params['ce_strike_offset'])}")
                    print(f"   Index Move Range:    ±{min(params['index_movement'])} - ±{max(params['index_movement'])}")
                    print(f"   Lot Size Range:      {min(params['lot_size'])} - {max(params['lot_size'])}")
                
                # Show full configuration example
                print("\n" + "="*100)
                print("📊 EXAMPLE: Full DTE-Specific Configuration")
                print("="*100)
                example_strategy = list(strategies.keys())[0]
                print(f"\nStrategy: {example_strategy}")
                print("\n" + strategies[example_strategy].to_string(index=False))
                
                print("\n" + "="*100)
                print("✅ Ready to run backtest with DTE-specific parameters!")
                print("💡 Each DTE will use its configured parameters during execution")
                print("="*100)
                
            except Exception as e:
                print(f"❌ Error generating configurations: {str(e)}")
                import traceback
                traceback.print_exc()
    
    select_all_btn.on_click(on_select_all)
    deselect_all_btn.on_click(on_deselect_all)
    preview_btn.on_click(on_preview_click)
    edit_btn.on_click(on_edit_click)
    generate_btn.on_click(on_generate_click)
    
    # Create checkbox grid
    checkbox_grid = []
    for i in range(0, len(preset_checkboxes), 2):
        preset_names = list(preset_checkboxes.keys())
        if i + 1 < len(preset_names):
            row = widgets.HBox([
                preset_checkboxes[preset_names[i]], 
                preset_checkboxes[preset_names[i+1]]
            ])
        else:
            row = widgets.HBox([preset_checkboxes[preset_names[i]]])
        checkbox_grid.append(row)
    
    checkbox_vbox = widgets.VBox(checkbox_grid)
    
    # Strategy descriptions with DTE info
    descriptions_html = """
    <div style='background: #f8f9fa; padding: 15px; border-radius: 5px; margin: 10px 0;'>
        <h4>Strategy Descriptions:</h4>
        <ul style='line-height: 1.8;'>
    """
    for name, preset in STRATEGY_PRESETS.items():
        descriptions_html += f"<li><strong>{name}:</strong> {preset['description']}</li>"
    descriptions_html += """
        </ul>
        <p style='color: #e67e22; font-weight: bold; margin-top: 10px;'>
            ⚡ NEW: Each strategy now has DTE-specific parameters that adjust as expiry approaches!
        </p>
    </div>
    """
    
    return widgets.VBox([
        widgets.HTML('<h3 style="color: #2c3e50;">⚙️ Step 2: Select & Configure Strategy Presets (DTE-Specific)</h3>'),
        widgets.HTML('<p style="color: #7f8c8d;">Choose strategies to test and customize DTE-specific parameters:</p>'),
        widgets.HTML(descriptions_html),
        widgets.HTML('<h4>Select Strategies:</h4>'),
        checkbox_vbox,
        widgets.HTML('<br>'),
        widgets.HBox([select_all_btn, deselect_all_btn]),
        widgets.HTML('<br>'),
        widgets.HTML('<h4>Configuration Actions:</h4>'),
        preview_btn,
        preview_output,
        widgets.HTML('<br>'),
        edit_btn,
        config_edit_output,
        widgets.HTML('<br>'),
        generate_btn,
        config_output
    ])

def create_backtest_execution_ui():
    """Create enhanced backtest execution interface with Entry DTE and Entry Time inputs"""
    
    # Entry DTE selection dropdown
    entry_dte_dropdown = widgets.Dropdown(
        options=['0 DTE', '1 DTE', '2 DTE', '3 DTE', '4 DTE', '5 DTE', '6 DTE'],
        value='3 DTE',
        description='Entry DTE:',
        style={'description_width': '120px'},
        layout=widgets.Layout(width='250px')
    )
    
    # Entry Time input
    entry_time_input = widgets.Text(
        value='09:15',
        placeholder='HH:MM (e.g., 09:15)',
        description='Entry Time:',
        style={'description_width': '120px'},
        layout=widgets.Layout(width='250px')
    )
    
    # Info message
    entry_params_info = widgets.HTML(
        value='''<div style="background: #e3f2fd; padding: 10px; border-radius: 5px; margin: 10px 0;">
            <strong>ℹ️ Entry Parameters:</strong> These settings will be applied consistently across all expiry files.
            The system will use the selected Entry DTE and Entry Time to determine strike prices for each file.
        </div>'''
    )
    
    run_btn = widgets.Button(
        description='🚀 Run Multi-Strategy Backtest',
        button_style='success',
        layout=widgets.Layout(width='300px', height='50px'),
        icon='play'
    )
    
    progress_output = widgets.Output()
    results_output = widgets.Output()
    
    def on_run_click(b):
        """Execute multi-strategy backtest with user-specified Entry DTE and Entry Time"""
        with progress_output:
            clear_output()
            
            if state.get_file_count() == 0:
                print("❌ Please upload files first!")
                return
            
            if len(state.strategy_configs) == 0:
                print("❌ Please generate strategy configurations first!")
                return
            
            # Get user inputs
            selected_entry_dte = entry_dte_dropdown.value
            selected_entry_time = entry_time_input.value.strip()
            
            # Validate entry time format
            try:
                datetime.strptime(selected_entry_time, '%H:%M')
            except ValueError:
                print("❌ Invalid time format! Please use HH:MM format (e.g., 09:15)")
                return
            
            print("="*80)
            print("🚀 STARTING MULTI-STRATEGY BACKTEST")
            print("="*80)
            print(f"📊 Files: {state.get_file_count()}")
            print(f"⚙️  Strategies: {len(state.strategy_configs)}")
            print(f"🔢 Total Combinations: {state.get_file_count() * len(state.strategy_configs)}")
            print(f"📍 Entry DTE: {selected_entry_dte}")
            print(f"⏰ Entry Time: {selected_entry_time}")
            print("="*80)
            
            def progress_callback(message):
                print(f"⏳ {message}")
            
            # Update strategy configs to use user-selected entry DTE
            # We need to modify the backtest to start from the selected DTE
            updated_strategy_configs = {}
            for strategy_name, config in state.strategy_configs.items():
                # Filter config to only include the selected entry DTE and below
                try:
                    selected_dte_idx = config[config['DTE'] == selected_entry_dte].index[0]
                    updated_config = config.iloc[selected_dte_idx:].copy()
                    updated_strategy_configs[strategy_name] = updated_config
                except IndexError:
                    print(f"⚠️  Warning: {strategy_name} does not have {selected_entry_dte} configuration. Skipping.")
                    continue
            
            if not updated_strategy_configs:
                print("❌ No valid strategy configurations for selected Entry DTE!")
                return
            
            # Store entry time in state for use in execute_strategy
            state.entry_time_override = selected_entry_time
            
            # Run backtest
            all_results = run_multi_strategy_backtest(
                state.uploaded_files,
                updated_strategy_configs,
                progress_callback
            )
            
            # Clear entry time override
            state.entry_time_override = None
            
            # Consolidate results
            results_df = consolidate_results(all_results)
            
            if results_df is None or len(results_df) == 0:
                print("\n❌ No trades executed")
                return
            
            state.consolidated_results = results_df
            
            # Display results
            with results_output:
                clear_output()
                
                print("\n" + "="*80)
                print("📊 MULTI-STRATEGY BACKTEST COMPLETED")
                print("="*80)
                print(f"📍 Entry DTE: {selected_entry_dte}")
                print(f"⏰ Entry Time: {selected_entry_time}")
                
                # Strategy Comparison
                comparison_df = generate_strategy_comparison(results_df)
                
                print("\n🏆 STRATEGY RANKINGS:")
                print("="*80)
                print(comparison_df[['Strategy', 'Total_PnL', 'Win_Rate_%', 'Profit_Factor', 
                                     'Sharpe_Ratio', 'Overall_Rank_Score']].to_string(index=False))
                
                print("\n\n📈 BEST PERFORMING STRATEGY:")
                print("="*80)
                best_strategy = comparison_df.iloc[0]
                print(f"Strategy: {best_strategy['Strategy']}")
                print(f"Total P&L: ₹{best_strategy['Total_PnL']:,.2f}")
                print(f"Win Rate: {best_strategy['Win_Rate_%']:.2f}%")
                print(f"Profit Factor: {best_strategy['Profit_Factor']}")
                print(f"Sharpe Ratio: {best_strategy['Sharpe_Ratio']:.2f}")
                print(f"Max Drawdown: ₹{best_strategy['Max_Drawdown']:,.2f}")
                
                # Export to Excel
                timestamp = datetime.now().strftime("%Y%m%d_%H%M%S")
                excel_filename = f'Multi_Strategy_Backtest_{selected_entry_dte.replace(" ", "")}_{selected_entry_time.replace(":", "")}_{timestamp}.xlsx'
                
                export_comprehensive_excel(results_df, updated_strategy_configs, excel_filename)
                
                print(f"\n💾 Results stored in 'state.consolidated_results'")
                print(f"📊 Excel report: {excel_filename}")
    
    run_btn.on_click(on_run_click)
    
    return widgets.VBox([
        widgets.HTML('<h3 style="color: #2c3e50;">🚀 Step 3: Execute Multi-Strategy Backtest</h3>'),
        widgets.HTML('<h4 style="color: #34495e; margin-top: 20px;">Entry Parameters:</h4>'),
        entry_params_info,
        widgets.HBox([entry_dte_dropdown, entry_time_input], layout=widgets.Layout(margin='10px 0')),
        widgets.HTML('<br>'),
        run_btn,
        progress_output,
        results_output
    ])

# ============================================================================
# MAIN UI ASSEMBLY
# ============================================================================

def create_main_ui():
    """Assemble enhanced main user interface"""
    
    header = widgets.HTML('''
        <div style="background: linear-gradient(135deg, #667eea 0%, #764ba2 100%); 
                    padding: 30px; 
                    border-radius: 10px; 
                    text-align: center;
                    margin-bottom: 20px;">
            <h1 style="color: white; margin: 0; font-size: 36px;">📈 Options Backtest System</h1>
            <p style="color: #e0e0e0; margin: 10px 0 0 0; font-size: 18px;">
                Multi-Strategy Optimization & Analysis Platform (WITH STRIKE CALCULATION FIX)
            </p>
        </div>
    ''')
    
    divider = widgets.HTML('<hr style="border: 2px solid #3498db; margin: 30px 0;">')
    
    file_upload_ui = create_enhanced_file_upload_ui()
    strategy_selection_ui = create_strategy_selection_ui()
    backtest_execution_ui = create_backtest_execution_ui()
    
    main_ui = widgets.VBox([
        header,
        file_upload_ui,
        divider,
        strategy_selection_ui,
        divider,
        backtest_execution_ui
    ])
    
    return main_ui

# ============================================================================
# LAUNCH APPLICATION
# ============================================================================

def launch_enhanced_backtest_system():
    """Launch the enhanced backtest system"""
    
    print("✅ Enhanced Options Backtest System Initialized")
    print("\n🎯 FEATURES:")
    print("   • Folder path batch processing")
    print("   • Multiple strategy presets (Conservative, Moderate, Aggressive, Scalper, Custom)")
    print("   • Multi-strategy parallel execution")
    print("   • Comprehensive strategy comparison & rankings")
    print("   • Strategy configuration tracking in Excel")
    print("   • Optimized file handling")
    print("\n📋 WORKFLOW:")
    print("   1. Load data files (upload or folder path)")
    print("   2. Select strategy presets to test")
    print("   3. Generate configurations")
    print("   4. Run multi-strategy backtest")
    print("   5. Analyze results & identify best strategies")
    print("\n" + "="*80 + "\n")
    
    main_ui = create_main_ui()
    display(main_ui)

# Auto-launch when imported
if __name__ == '__main__':
    launch_enhanced_backtest_system()

✅ Enhanced Options Backtest System Initialized (WITH CRITICAL FIX)

⚡ STRIKE CALCULATION FIX APPLIED:
   • Each expiry file now independently calculates PE/CE strikes
   • Strikes based on actual index close from file data at entry time
   • No more duplicate strikes across different expiry files

🎯 FEATURES:
   • Folder path batch processing
   • Multiple strategy presets (Conservative, Moderate, Aggressive, Scalper, Custom)
   • Multi-strategy parallel execution
   • Comprehensive strategy comparison & rankings
   • Strategy configuration tracking in Excel
   • Optimized file handling

📋 WORKFLOW:
   1. Load data files (upload or folder path)
   2. Select strategy presets to test
   3. Generate configurations
   4. Run multi-strategy backtest
   5. Analyze results & identify best strategies


